In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dropout, Dense, LSTM, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from plot import plot_training_vs_validation
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
bin_data = pd.read_csv("bin_data.csv")

In [7]:
X = bin_data.drop(columns = ['label'], axis = 1)
Y = bin_data['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 50)

In [10]:
X_train_reshaped = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

In [11]:
X_test_reshaped = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [13]:
unique_labels = np.unique(y_train)
num_classes = len(unique_labels)

In [14]:
unique_labels, num_classes

(array([0, 1], dtype=int64), 2)

In [18]:
cnn_model = Sequential()
cnn_model.add(Conv1D(filters = 64, kernel_size = 3, activation = 'relu', input_shape = (X_train.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size = 2))
cnn_model.add(Dropout(0.35))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation = 'relu'))
cnn_model.add(Dense(num_classes, activation='softmax'))

In [24]:
adam_optimizer = Adam(learning_rate = 1e-4)
cnn_model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = adam_optimizer,
    metrics = ['accuracy']
)
callbacks = [
    EarlyStopping(monitor = 'val_loss', patience = 10),
    ModelCheckpoint(
        filepath = 'best_model_cnn.h5',
        monitor = 'val_loss',
        save_best_only = True
    )
]

In [25]:
cnn_history = cnn_model.fit(
    X_train_reshpaed, y_train, epochs = 50, batch_size = 256, validation_data = (X_test_reshaped, y_test), callbacks=callbacks
)

Epoch 1/50
364/364 [==============================] - 5s 8ms/step - loss: 0.2364 - accuracy: 0.9413 - val_loss: 0.1114 - val_accuracy: 0.9616
Epoch 2/50
364/364 [==============================] - 3s 8ms/step - loss: 0.1492 - accuracy: 0.9496 - val_loss: 0.1394 - val_accuracy: 0.9593
Epoch 3/50
364/364 [==============================] - 3s 8ms/step - loss: 0.1609 - accuracy: 0.9483 - val_loss: 0.1152 - val_accuracy: 0.9633
Epoch 4/50
364/364 [==============================] - 3s 8ms/step - loss: 0.1677 - accuracy: 0.9479 - val_loss: 0.1268 - val_accuracy: 0.9593
Epoch 5/50
364/364 [==============================] - 3s 8ms/step - loss: 0.1734 - accuracy: 0.9483 - val_loss: 0.1531 - val_accuracy: 0.9512
Epoch 6/50
364/364 [==============================] - 3s 7ms/step - loss: 0.1521 - accuracy: 0.9497 - val_loss: 0.1589 - val_accuracy: 0.9366
Epoch 7/50
364/364 [==============================] - 2s 7ms/step - loss: 0.1513 - accuracy: 0.9504 - val_loss: 0.1303 - val_accuracy: 0.9594
Epoch 

In [26]:
y_test_probabilities = cnn_model.predict(X_test_reshaped)
y_test_pred = np.argmax(y_test_probabilities, axis = 1)

728/728 [==============================] - 3s 2ms/step


In [27]:
conf_matrix = confusion_matrix(y_test, y_test_pred)

In [31]:
print(conf_matrix)

[[17009   386]
 [  445  5431]]


In [32]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     17395
           1       0.93      0.92      0.93      5876

    accuracy                           0.96     23271
   macro avg       0.95      0.95      0.95     23271
weighted avg       0.96      0.96      0.96     23271



In [1]:
print()